In [20]:
#r "nuget:Microsoft.ML,1.4.0"
#r "nuget:Microsoft.Data.Analysis,0.3.0"
#r "nuget:Microsoft.ML.OnnxTransformer,1.4.0"
#r "nuget:Microsoft.ML.OnnxConverter,0.16.0"
#r "nuget:Microsoft.ML.OnnxRuntime,1.1.0"

Installed package Microsoft.ML.OnnxTransformer version 1.4.0

Installed package Microsoft.ML version 1.4.0

Installed package Microsoft.ML.OnnxConverter version 0.16.0

Installed package Microsoft.ML.OnnxRuntime version 1.1.0

Installed package Microsoft.Data.Analysis version 0.3.0

In [21]:
using System.IO;
using Microsoft.Data.Analysis;
using XPlot.Plotly;

In [22]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

In [23]:
var data = DataFrame.LoadCsv("./SalaryData.csv");

In [5]:
data

0,1.1,39343
1,1.3,46205
2,1.5,37731
3,2,43525
4,2.2,39891
5,2.9,56642
6,3,60150
7,3.2,54445
8,3.2,64445
9,3.7,57189
10,3.9,63218


In [24]:
data.Description()

0,Length (excluding null values),30,30
1,Max,10.5,122391
2,Min,1.1,37731
3,Mean,5.3133335,76003


In [25]:
static T[] Shuffle<T>(T[] array)
{
    Random rand = new Random();
    for (int i = 0; i < array.Length; i++)
    {
        int r = i + rand.Next(array.Length - i);
        T temp = array[r];
        array[r] = array[i];
        array[i] = temp;
    }
    return array;
}

int[] randomIndices = Shuffle(Enumerable.Range(0, (int)data.Rows.Count).ToArray());
int testSize = (int)(data.Rows.Count * .1);
int[] trainRows = randomIndices[testSize..];
int[] testRows = randomIndices[..testSize];

DataFrame data_train = data[trainRows];
DataFrame data_test = data[testRows];

display(data_train.Rows.Count);
display(data_test.Rows.Count);

27

3

In [8]:
using Microsoft.ML;

In [9]:
var context = new MLContext();

###Create Pipeline

In [26]:
var pipeline = context.Transforms.Concatenate("Features", "yearsExperience")
                .Append(context.Regression.Trainers.Sdca(labelColumnName: "salary"));

In [27]:
ITransformer model = pipeline.Fit(data_train);

In [28]:
var testResults = model.Transform(data_test);

var metrics = context.Regression.Evaluate(testResults, labelColumnName: "salary", scoreColumnName: "Score");

In [29]:
Console.WriteLine($"*************************************************");
Console.WriteLine($"*       Metrics for regression model      ");
Console.WriteLine($"*------------------------------------------------");
Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
Console.WriteLine($"*************************************************");

*************************************************
*       Metrics for regression model      
*------------------------------------------------
*       LossFn:        34047799
*       R2 Score:      0.94
*       Absolute loss: 5667.39
*       Squared loss:  34047799.12
*       RMS loss:      5835.05
*************************************************


In [30]:
private static readonly string MODEL_NAME = "model.onnx";

In [31]:
using System.IO;

using (var stream = File.Create(MODEL_NAME))
{
    context.Model.ConvertToOnnx(model, data, stream);
}

In [32]:
using Microsoft.ML.OnnxRuntime;
using Microsoft.ML.OnnxRuntime.Tensors;

In [33]:
private static NamedOnnxValue GetNamedOnnxValue<T>(IReadOnlyDictionary<string, NodeMetadata> inputMeta, string column, T value)
{
    T[] inputDataInt = new T[] { value };
    var tensor = new DenseTensor<T>(inputDataInt, inputMeta[column].Dimensions);
    var namedOnnxValue = NamedOnnxValue.CreateFromTensor<T>(column, tensor);
    return namedOnnxValue;
}

In [34]:
var session = new InferenceSession(MODEL_NAME);

var inputMeta = session.InputMetadata;

var container = new List<NamedOnnxValue>();
container.Add(GetNamedOnnxValue<float>(inputMeta, "yearsExperience", 1.1f));
container.Add(GetNamedOnnxValue<float>(inputMeta, "salary", 0f));


In [35]:
var result = session.Run(container);
var output = session.Run(container).First(x => x.Name == "Score0").AsTensor<float>().Max();

Console.WriteLine($"**********************************************************************");
Console.WriteLine($"Predicted salary: {output:0.####}, actual salary: 39343.0");
Console.WriteLine($"**********************************************************************");


**********************************************************************
Predicted salary: 37134.17, actual salary: 39343.0
**********************************************************************
